<a href="https://colab.research.google.com/github/Aishriya30/Drowsiness-Alert/blob/main/Neuro_Guard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:

df = pd.read_csv('/go_emotions_dataset.csv')
df.head()


,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,eemcysk,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df.describe()

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
count,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,...,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000
mean,0.081103,0.043768,0.038272,0.064472,0.083418,0.028401,0.034840,0.045885,0.018071,0.040095,...,0.038779,0.008569,0.041259,0.006164,0.041591,0.006102,0.011954,0.031994,0.026105,0.261797
std,0.272994,0.204580,0.191853,0.245592,0.276514,0.166116,0.183374,0.209236,0.133208,0.196182,...,0.193067,0.092172,0.198890,0.078269,0.199653,0.077880,0.108679,0.175985,0.159448,0.439614
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
check_null=df.isnull().sum()
check_null

,0
id,0
text,0
example_very_unclear,0
admiration,0
amusement,0
anger,0
annoyance,0
approval,0
caring,0
confusion,0


In [ ]:
df.drop(columns=['id', 'example_very_unclear'], inplace=True, errors='ignore')

In [ ]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"@\w+|\#", '', text)
    text = re.sub(r"[^A-Za-z0-9\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)
df['text'].head()


,text
0,that game hurt
1,sexuality shouldnt be a grouping category it m...
2,you do right if you dont care then fuck em
3,man i love reddit
4,name was nowhere near them he was by the falcon


In [ ]:
emotion_cols = [col for col in df.columns if col not in ['text']]
scaler = StandardScaler()
df[emotion_cols] = scaler.fit_transform(df[emotion_cols])


In [ ]:
df['dominant_emotion'] = df[emotion_cols].idxmax(axis=1)


In [ ]:
print(df.shape)

(211225, 30)


In [ ]:
negative_emotions = ['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness']

In [ ]:
df['total_negative'] = df[negative_emotions].sum(axis=1)

In [ ]:

np.random.seed(42)
df['hour_of_day'] = np.random.randint(0, 24, size=len(df))

def circadian_alertness(hour):
    if 2 <= hour <= 6:
        return 0.2
    elif 7 <= hour <= 9 or 18 <= hour <= 22:
        return 0.5
    elif 10 <= hour <= 17:
        return 1.0
    else:
        return 0.3

df['alertness_level'] = df['hour_of_day'].apply(circadian_alertness)


In [ ]:
df['adjusted_negative'] = df['total_negative'] * (1 - df['alertness_level'])



In [ ]:
THRESHOLD = 0.5
df['block_output'] = (df['adjusted_negative'] > THRESHOLD).astype(int)

In [ ]:
X_text = df['text']
y_emotions = df[emotion_cols]

vectorizer = TfidfVectorizer(max_features=3000)
X_vec = vectorizer.fit_transform(X_text)
binary_emotion_df = df[emotion_cols].apply(lambda x: (x > 0).astype(int))

X_train, X_test, y_train, y_test = train_test_split(X_vec, binary_emotion_df, test_size=0.2, random_state=42)

emotion_model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
emotion_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

In [ ]:
X_block = df.drop(columns=['text', 'dominant_emotion', 'block_output'])
y_block = df['block_output']

block_model = Sequential([
    Dense(32, activation='relu', input_shape=(X_block.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
block_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
block_model.fit(X_block, y_block, epochs=10, batch_size=16, validation_split=0.2)


Epoch 1/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 1.0000 - loss: 1.0333e-05 - val_accuracy: 1.0000 - val_loss: 1.2825e-06
Epoch 2/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - accuracy: 1.0000 - loss: 7.6345e-08 - val_accuracy: 1.0000 - val_loss: 1.5032e-09
Epoch 3/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - accuracy: 1.0000 - loss: 6.1562e-08 - val_accuracy: 1.0000 - val_loss: 8.6243e-09
Epoch 4/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 1.0000 - loss: 2.9114e-10 - val_accuracy: 1.0000 - val_loss: 1.8732e-10
Epoch 5/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - accuracy: 1.0000 - loss: 2.4115e-11 - val_accuracy: 1.0000 - val_loss: 1.2941e-10
Epoch 6/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 44s 3ms/step - accuracy: 1.0000 - loss: 1.5404e-11 - val_accuracy: 1.0000 - val_loss: 1.2629e-10
Epoch 7/10
10562/10562 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 1.0000 - loss: 1.4074e-11 - val_accuracy: 1.0000 - val_loss: 7.9409e-11
Epoch 8/10
10

In [ ]:
def full_prediction_pipeline(text, hour_of_day=9):
    clean = clean_text(text)
    vec = vectorizer.transform([clean])
    emotion_scores = emotion_model.predict(vec)[0]

    # Create emotion score dict
    emotion_score_dict = dict(zip(emotion_cols, emotion_scores))

    # Calculate total negative
    total_negative = sum([emotion_score_dict[e] for e in negative_emotions])

    # Get alertness level
    alertness = circadian_alertness(hour_of_day)

    # Adjusted inhibitory score
    adjusted_negative = total_negative * (1 - alertness)

    # Block or not
    block = 1 if adjusted_negative > THRESHOLD else 0

    # Dominant emotion
    dominant_emotion = max(emotion_score_dict, key=emotion_score_dict.get)

    return {
        'dominant_emotion': dominant_emotion,
        'emotion_scores': emotion_score_dict,
        'total_negative': total_negative,
        'alertness_level': alertness,
        'adjusted_negative': adjusted_negative,
        'block_output': block
    }



In [ ]:
sample = "I feel happy, pain funny, joyful today"
result = full_prediction_pipeline(sample, hour_of_day=3)

print("Sample Text:", sample)
print("Dominant Emotion:", result['dominant_emotion'])
print("Adjusted Negative Score:", result['adjusted_negative'])
print("Block Output:", "BLOCKED " if result['block_output'] else "ALLOWED ")
print("Top Emotions:", {k: v for k, v in result['emotion_scores'].items() if v > 0})



Sample Text: I feel happy, pain funny, joyful today
Dominant Emotion: joy
Adjusted Negative Score: 0.0
Block Output: ALLOWED 
Top Emotions: {'joy': np.int64(1)}
